In [1]:
import numpy as np
import pandas as pd
import requests as rq

from plotnine import *

In [2]:
import itertools as it
import functools as ft

def map_list(*args, **kwargs):
    return list(map(*args, **kwargs))

def filter_list(*args, **kwargs):
    return list(filter(*args, **kwargs))

def map_dfr(*args, **kwargs):
    return pd.concat(map_list(*args, **kwargs))

In [3]:
DATA_DIR = '../../data/'

The EIA also provides six month bulk download files. Lets get them into one data set for Southern Company

In [4]:
files = [f'{DATA_DIR}EIA/EIA930_BALANCE_2023_Jan_Jun.csv',
         f'{DATA_DIR}EIA/EIA930_BALANCE_2023_Jul_Dec.csv',
         f'{DATA_DIR}EIA/EIA930_BALANCE_2024_Jan_Jun.csv']

def clean_columns(cols):
    clean_cols = list(map(lambda x: x.replace(' ', '_'), cols))
    clean_cols = list(map(lambda x: x.lower(), clean_cols))
    clean_cols = list(map(lambda x: x.replace('(', ''), clean_cols))
    clean_cols = list(map(lambda x: x.replace(')', ''), clean_cols))
    return clean_cols

def read_soco_rows_only(filename):
    df = pd.read_csv(filename)
    clean_cols = clean_columns(df.columns)
    df.columns = clean_cols
    clean_df = df.query('balancing_authority == "SOCO"')
    return clean_df

power_df = map_dfr(read_soco_rows_only, files)
print(power_df.shape)
power_df.head()

/var/folders/hs/y8p4rwwn25n29b8gnr8l8xdh0000gn/T/ipykernel_74699/1930422923.py:13: DtypeWarning: Columns (5,6,7,8,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
/var/folders/hs/y8p4rwwn25n29b8gnr8l8xdh0000gn/T/ipykernel_74699/1930422923.py:13: DtypeWarning: Columns (5,6,7,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.


(10320, 42)


/var/folders/hs/y8p4rwwn25n29b8gnr8l8xdh0000gn/T/ipykernel_74699/1930422923.py:13: DtypeWarning: Columns (19,21,23,25,26,27,29,30,31,36,38,40) have mixed types. Specify dtype option on import or set low_memory=False.


,balancing_authority,data_date,hour_number,local_time_at_end_of_hour,utc_time_at_end_of_hour,demand_forecast_mw,demand_mw,net_generation_mw,total_interchange_mw,sumvalid_dibas_mw,...,net_generation_mw_from_other_fuel_sources_imputed,net_generation_mw_from_coal_adjusted,net_generation_mw_from_natural_gas_adjusted,net_generation_mw_from_nuclear_adjusted,net_generation_mw_from_all_petroleum_products_adjusted,net_generation_mw_from_hydropower_and_pumped_storage_adjusted,net_generation_mw_from_solar_adjusted,net_generation_mw_from_wind_adjusted,net_generation_mw_from_other_fuel_sources_adjusted,region
204129,SOCO,01/01/2023,1,01/01/2023 1:00:00 AM,01/01/2023 7:00:00 AM,"18,351","19,110","19,480",370,370,...,NaN,"2,674","10,256","5,859",-1.0,349,1,0.0,342.0,SE
204130,SOCO,01/01/2023,2,01/01/2023 2:00:00 AM,01/01/2023 8:00:00 AM,"17,864","18,814","19,158",344,344,...,NaN,"2,671","9,951","5,860",0.0,335,1,0.0,340.0,SE
204131,SOCO,01/01/2023,3,01/01/2023 3:00:00 AM,01/01/2023 9:00:00 AM,"17,688","18,482","18,927",445,445,...,NaN,"2,658","9,739","5,858",-2.0,339,1,0.0,334.0,SE
204132,SOCO,01/01/2023,4,01/01/2023 4:00:00 AM,01/01/2023 10:00:00 AM,"17,660","18,293","18,837",544,544,...,NaN,"2,636","9,688","5,859",0.0,326,1,0.0,328.0,SE
204133,SOCO,01/01/2023,5,01/01/2023 5:00:00 AM,01/01/2023 11:00:00 AM,"18,039","18,485","18,992",507,507,...,NaN,"2,634","9,799","5,859",0.0,375,-1,0.0,326.0,SE


In [5]:
power_df.dtypes

balancing_authority                                               object
data_date                                                         object
hour_number                                                        int64
local_time_at_end_of_hour                                         object
utc_time_at_end_of_hour                                           object
demand_forecast_mw                                                object
demand_mw                                                         object
net_generation_mw                                                 object
total_interchange_mw                                              object
sumvalid_dibas_mw                                                 object
demand_mw_imputed                                                 object
net_generation_mw_imputed                                         object
total_interchange_mw_imputed                                     float64
demand_mw_adjusted                                 

In [6]:
power_df.isnull().sum(axis = 0)

balancing_authority                                                  0
data_date                                                            0
hour_number                                                          0
local_time_at_end_of_hour                                            0
utc_time_at_end_of_hour                                              0
demand_forecast_mw                                                   1
demand_mw                                                           24
net_generation_mw                                                   24
total_interchange_mw                                                24
sumvalid_dibas_mw                                                   48
demand_mw_imputed                                                10320
net_generation_mw_imputed                                        10320
total_interchange_mw_imputed                                     10320
demand_mw_adjusted                                                  24
net_ge

In [7]:
power_df.loc[(power_df['net_generation_mw_from_solar_adjusted'] != power_df['net_generation_mw_from_solar']) & (~power_df['net_generation_mw_from_solar'].isnull()),
             ['data_date', 'hour_number', 'demand_forecast_mw', 'net_generation_mw', 'net_generation_mw_from_solar', 'net_generation_mw_from_solar_adjusted']].head(50)


,data_date,hour_number,demand_forecast_mw,net_generation_mw,net_generation_mw_from_solar,net_generation_mw_from_solar_adjusted


In [8]:
non_adj_imp_cols = map_list(lambda x: ('adjusted' not in x) & ('imputed' not in x), power_df.columns)
power_df_clean = power_df.loc[:, non_adj_imp_cols].copy()
power_df_clean.shape

(10320, 20)

In [9]:
power_df_clean['date'] = pd.to_datetime(power_df_clean['data_date'])
power_df_clean['day_of_week'] = power_df_clean['date'].dt.dayofweek
power_df_clean['day_of_year'] = power_df_clean['date'].dt.dayofyear
power_df_clean['year'] = power_df_clean['date'].dt.year
power_df_clean['month'] = power_df_clean['date'].dt.month

In [10]:
mw_cols = filter_list(lambda x: 'mw' in x, power_df_clean.columns)
col_dtypes = power_df_clean.dtypes.to_dict()
chr_cols = filter_list(lambda c: col_dtypes[c] == 'object', mw_cols)

print(f'converting {len(chr_cols)} to numeric')
for c in chr_cols:
    power_df_clean[c] = pd.to_numeric(power_df_clean[c].astype('str').str.replace(',', ''), errors = 'coerce')

converting 13 to numeric


In [11]:
better_net_gen_cols = power_df_clean.columns.str.replace('net_generation_mw_from_', '')
better_net_gen_cols = better_net_gen_cols.str.replace('_and_pumped_storage', '')
power_df_clean.columns = better_net_gen_cols
power_df_clean.head(10)

,balancing_authority,data_date,hour_number,local_time_at_end_of_hour,utc_time_at_end_of_hour,demand_forecast_mw,demand_mw,net_generation_mw,total_interchange_mw,sumvalid_dibas_mw,...,solar,wind,other_fuel_sources,unknown_fuel_sources,region,date,day_of_week,day_of_year,year,month
204129,SOCO,01/01/2023,1,01/01/2023 1:00:00 AM,01/01/2023 7:00:00 AM,18351.0,19110.0,19480.0,370.0,370.0,...,1.0,0.0,342.0,NaN,SE,2023-01-01,6,1,2023,1
204130,SOCO,01/01/2023,2,01/01/2023 2:00:00 AM,01/01/2023 8:00:00 AM,17864.0,18814.0,19158.0,344.0,344.0,...,1.0,0.0,340.0,NaN,SE,2023-01-01,6,1,2023,1
204131,SOCO,01/01/2023,3,01/01/2023 3:00:00 AM,01/01/2023 9:00:00 AM,17688.0,18482.0,18927.0,445.0,445.0,...,1.0,0.0,334.0,NaN,SE,2023-01-01,6,1,2023,1
204132,SOCO,01/01/2023,4,01/01/2023 4:00:00 AM,01/01/2023 10:00:00 AM,17660.0,18293.0,18837.0,544.0,544.0,...,1.0,0.0,328.0,NaN,SE,2023-01-01,6,1,2023,1
204133,SOCO,01/01/2023,5,01/01/2023 5:00:00 AM,01/01/2023 11:00:00 AM,18039.0,18485.0,18992.0,507.0,507.0,...,-1.0,0.0,326.0,NaN,SE,2023-01-01,6,1,2023,1
204134,SOCO,01/01/2023,6,01/01/2023 6:00:00 AM,01/01/2023 12:00:00 PM,18500.0,18807.0,19211.0,404.0,404.0,...,0.0,0.0,332.0,NaN,SE,2023-01-01,6,1,2023,1
204135,SOCO,01/01/2023,7,01/01/2023 7:00:00 AM,01/01/2023 1:00:00 PM,19316.0,18948.0,19426.0,478.0,478.0,...,1.0,0.0,350.0,NaN,SE,2023-01-01,6,1,2023,1
204136,SOCO,01/01/2023,8,01/01/2023 8:00:00 AM,01/01/2023 2:00:00 PM,20345.0,19238.0,19626.0,388.0,388.0,...,155.0,0.0,346.0,NaN,SE,2023-01-01,6,1,2023,1
204137,SOCO,01/01/2023,9,01/01/2023 9:00:00 AM,01/01/2023 3:00:00 PM,20877.0,20035.0,20412.0,377.0,377.0,...,548.0,0.0,343.0,NaN,SE,2023-01-01,6,1,2023,1
204138,SOCO,01/01/2023,10,01/01/2023 10:00:00 AM,01/01/2023 4:00:00 PM,20928.0,20410.0,20734.0,324.0,324.0,...,1182.0,0.0,348.0,NaN,SE,2023-01-01,6,1,2023,1


In [12]:
better_net_gen_cols

Index(['balancing_authority', 'data_date', 'hour_number',
       'local_time_at_end_of_hour', 'utc_time_at_end_of_hour',
       'demand_forecast_mw', 'demand_mw', 'net_generation_mw',
       'total_interchange_mw', 'sumvalid_dibas_mw', 'coal', 'natural_gas',
       'nuclear', 'all_petroleum_products', 'hydropower', 'solar', 'wind',
       'other_fuel_sources', 'unknown_fuel_sources', 'region', 'date',
       'day_of_week', 'day_of_year', 'year', 'month'],
      dtype='object')

In [13]:
power_df_tall = power_df_clean.melt(
    id_vars = ['date', 'day_of_week', 'day_of_year', 'year', 'month', 'hour_number'],
    value_vars = ['coal', 'natural_gas', 'nuclear', 'all_petroleum_products', 'hydropower', 'solar', 'wind'],
    var_name = 'source', value_name = 'mw_generated'
)
print(power_df_tall.shape)
power_df_tall.head()

(72240, 8)


,date,day_of_week,day_of_year,year,month,hour_number,source,mw_generated
0,2023-01-01,6,1,2023,1,1,coal,2674.0
1,2023-01-01,6,1,2023,1,2,coal,2671.0
2,2023-01-01,6,1,2023,1,3,coal,2658.0
3,2023-01-01,6,1,2023,1,4,coal,2636.0
4,2023-01-01,6,1,2023,1,5,coal,2634.0


In [14]:
power_df_clean.to_parquet(f'{DATA_DIR}EIA/soco_power_gen.parquet')
power_df_tall.to_parquet(f'{DATA_DIR}EIA/soco_power_gen_tall.parquet')